# Transpile any model

Transpile a `Keras` model into a `PyTorch` module.

⚠️ If you are running this notebook in Colab, you will have to install `Ivy` and some dependencies manually. You can do so by running the cell below ⬇️

If you want to run the notebook locally but don't have Ivy installed just yet, you can check out the [Get Started section of the docs.](https://unify.ai/docs/ivy/overview/get_started.html)

In [1]:
!pip install ivy

As we've already seen, `ivy.transpile` can convert functions and whole libraries from one framework to another. However, in machine learning and deep learning, much of the focus is on trainable modules. Fortunately, Ivy can manage the parameters of these modules and ensure that the transpiled module is fully compatible with the target framework. This allows you to take full advantage of the training utilities provided by any framework and to build complex models on top of the transpiled ones. Let's see how this works!

Let's start by importing the neccessary libraries:

In [2]:
import ivy
import torch
import numpy as np
import tensorflow as tf
tf.keras.utils.set_random_seed(0)
torch.manual_seed(0)

2023-11-01 16:31:42.301436: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


There are examples which use more involved models in the [Guides](https://lets-unify.ai/demos#guides) section, but to keep things simple, let's define a basic convolutional network using Keras' Sequential API to use it as the starting point!

In [3]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 3)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10, activation='softmax')
])

Now, we can use `ivy.transpile` to convert this `Keras` model to `PyTorch`. Since we are passing a framework-specific object to the `transpile` function, there is no need to specify the `source` keyword argument this time.

In [4]:
input_array = tf.random.normal((1, 28, 28, 3))
torch_model = ivy.transpile(model, source = "tensorflow", to="torch", args=(input_array,))

/workspaces/ivy/ivy/utils/exceptions.py:390: UserWarning: The current backend: 'tensorflow' does not support inplace updates natively. Ivy would quietly create new arrays when using inplace updates with this backend, leading to memory overhead (same applies for views). If you want to control your memory management, consider doing ivy.set_inplace_mode('strict') which should raise an error whenever an inplace update is attempted with this backend.
  warnings.warn(
/workspaces/ivy/ivy/compiler/compiler.py:159: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  return _transpile(
/workspaces/ivy/ivy/compiler/compiler.py:159: UserWarning: `layer.updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  return _transpile(


Thanks to (eager) transpilation, we now have a fully-fledged `torch.nn.module` 🚀

In [5]:
isinstance(torch_model, torch.nn.Module)

True

This means that we can pass `PyTorch` inputs (keeping the channels-last format of `Keras`, as the new computational graph is identical to the original one!) and get `PyTorch` tensors as the output:

In [6]:
input_array = torch.rand((1, 28, 28, 3))
output_array = torch_model(input_array)
print(output_array)

tensor([[0.0893, 0.1504, 0.1372, 0.0991, 0.0867, 0.0851, 0.0911, 0.0804, 0.0926,
         0.0881]], grad_fn=<SoftmaxBackward0>)


Furthermore, having a `torch.nn.Module` also enables you to train the model using PyTorch training code, and also to use the transpiled model to build more complex torch models, as shown in the [Transpiling a haiku model to build on top](../guides/02_transpiling_a_haiku_model.ipynb) guide!

## Round up

This is the last tutorial related to the tracer/transpiler in [Learn the Basics](https://lets-unify.ai/demos#learn-the-basics). In the next tutorial, we'll go over an introduction to building models directly using Ivy 👨‍💻. If you are interested in continuing to learn about transpilation, you can check out the more complex tutorials in the [Guides](https://lets-unify.ai/demos#guides) section!